In [1]:
# 케라스  API를 사용 패션MNIST 데이터셋 불러오기
import keras
(train_input, train_target), (test_input, test_target) =\
    keras.datasets.fashion_mnist.load_data()

# 픽셀값 변환 및 1차원 배열로 바꾸기, 사이킷런으로 훈련세트와 검증세트로 나누기
from sklearn.model_selection import train_test_split

train_scaled = train_input/255.0
train_scaled = train_scaled.reshape(-1,28*28)
train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled, train_target, test_size=0.2, random_state=42)

2025-08-24 15:10:04.734445: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-24 15:10:04.866561: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-24 15:10:04.866662: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-24 15:10:04.867586: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-24 15:10:04.934526: I tensorflow/core/platform/cpu_feature_g

In [2]:
# 입력층과 출력층 사이에 밀집층 추가
# 입력층과 출력층 사이에 있는 모든 층을 은닉층이라고 한다.
# 은닉층의 활성화 함수는 비교적 자유롭다(출력층은 시그모이드함수, 소프트맥스함수)
# 은닉층에 활성화 함수가 필요한이유? -> 선형만 계산하면 아무리 층을 쌓아도 결국 하나의 선형모델이 돼버린다 ->  복잡한 문제를 풀지 못함
# 따라서 선형계산을 비선형적으로 만들어주기 위해서

# 시그모이드 활성화 함수를 사용한 은닉층과 소프트 맥스 함수를 사용한 출력층을 케라스의 Dense 클래스로 만들기
inputs = keras.layers.Input(shape=(784,))
dense1 = keras.layers.Dense(100, activation='sigmoid') #100개의 뉴런
dense2 = keras.layers.Dense(10, activation='softmax') #10개의 뉴런

In [3]:
# 심층 신경망 만들기
model = keras.Sequential([inputs, dense1, dense2]) # 입력층을 맨앞에, 출력층을 맨뒤에

2025-08-24 15:10:07.191780: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-08-24 15:10:07.249920: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-08-24 15:10:07.250091: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [4]:
# summary() 메서드를 호출하면 층에 대한 정보확인 가능
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               78500     
                                                                 
 dense_1 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79510 (310.59 KB)
Trainable params: 79510 (310.59 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [5]:
# 첫 번째 차원은 샘플 개수를 나타내지만, 미니배치경사하강법을 이용하여 나누어 데이터를 사용하기 때문에 None으로 정의 -> 배치차원
# None-trainable params -> 경사하강법으로 훈련되지 않은 파라미터 개수

In [6]:
# 층을 추가하는 다른 방법
model = keras.Sequential([
   keras.layers.Input(shape=(784,)),
   keras.layers.Dense(100, activation='sigmoid', name='hidden_layer'),
   keras.layers.Dense(10, activation='softmax', name='output_layer')
   ], name='패션 MNIST 모델') 

model.summary()
# Seqeuential 클래스 생성자가 매우 길어짐, 조건에 따라 층을 추가 할 수 없을 수 있다.

Model: "패션 MNIST 모델"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden_layer (Dense)        (None, 100)               78500     
                                                                 
 output_layer (Dense)        (None, 10)                1010      
                                                                 
Total params: 79510 (310.59 KB)
Trainable params: 79510 (310.59 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [7]:
# add() 메서드를 호출하여 층을 추가하는 방법
model = keras.Sequential()
model.add(keras.layers.Input(shape=(784,)))
model.add(keras.layers.Dense(100, activation='sigmoid'))
model.add(keras.layers.Dense(10, activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 100)               78500     
                                                                 
 dense_3 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79510 (310.59 KB)
Trainable params: 79510 (310.59 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [8]:
# 5번의 에포크 동안 훈련
model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_scaled, train_target, epochs=5)
# 인공신경망에 몇 개의 층을 추가하더라도 compile() 메서드와 fit() 메서드의 사용법은 동일 -> 케라스API의 장점

Epoch 1/5


2025-08-24 15:10:08.268762: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x70f2ee408420 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-08-24 15:10:08.268779: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3080 Ti Laptop GPU, Compute Capability 8.6
2025-08-24 15:10:08.273067: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-08-24 15:10:08.284239: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8904
2025-08-24 15:10:08.334103: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1500/1500 [==============================] - 5s 3ms/step - loss: 0.5684 - accuracy: 0.8063
Epoch 2/5
1500/1500 [==============================] - 4s 2ms/step - loss: 0.4106 - accuracy: 0.8515
Epoch 3/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3746 - accuracy: 0.8643
Epoch 4/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3523 - accuracy: 0.8711
Epoch 5/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3348 - accuracy: 0.8792


In [9]:
# 시그모이드 함수는 층이 많을 수록 (그래프가 누워있어) 그 효과가 누적되어 학습을 어렵게 함
# 렐루함수 -> z가 0보다 크면 z를 출력, 0보다 작으면 0을 출력
# 렐루 함수는 특히 이미지 처리에서 좋은 성능을 냄 

In [10]:
# Flatten 클래스는 배치 차원을 제외하고 나머지 입력 차원을 모두 일렬로 펼치는 역할만 한다.
# ex. (32, 28, 28) ->(32, 784), 즉 32장은 그대로 유지, 각이미지를 1D 벡터로 펼친것
# 배치차원은 아래와 같이 입력층 바로 뒤에 추가
model = keras.Sequential()
model.add(keras.layers.Input(shape=(28,28)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

model.summary()
# 784개의 입력이 첫 번째 은닉층에 전달

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense_4 (Dense)             (None, 100)               78500     
                                                                 
 dense_5 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79510 (310.59 KB)
Trainable params: 79510 (310.59 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [11]:
# 훈련데이터를 다시 준비 및 훈련
(train_input, train_target), (test_input, test_target) =\
    keras.datasets.fashion_mnist.load_data()
train_scaled = train_input/255.0
train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled, train_target, test_size=0.2, random_state=42)

model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.5280 - accuracy: 0.8140
Epoch 2/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3895 - accuracy: 0.8605
Epoch 3/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3505 - accuracy: 0.8744
Epoch 4/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3311 - accuracy: 0.8808
Epoch 5/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3150 - accuracy: 0.8869


In [12]:
# 검증세트에서의 성능 확인
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 1s 1ms/step - loss: 0.3980 - accuracy: 0.8631


[0.39797794818878174, 0.8630833625793457]

In [13]:
# 옵티마이저 -> 케라스의 다양한 종류의 경사하강법 알고리즘
# 기본 경사하강법 옵티마이저 ->SGD 클래스에서 제공, momentum 매개변수의 기본값은 0
# 0보다 큰값으로 지정하면 모멘텀 최적화를 사용(보통 0.9이상을 지정)
# SGD 클래스의 nesterov 매개변수를 기본값 False에서 True로 바꾸면 네스테로프 모멘텀 최적화를 사용, ex. sgd = keras.optimizers.SGD(momentum=0.9, nesterov=True)
# 네스테로프 모멘텀은 모멘텀 최적화를 2번 반복하여 구현
# 모델이 최적점에 가까이 갈수록 학습률을 낮출 수 있다. -> 적응적 학습률
# 적응적 학습률을 사용하는 대표적인 optimizer -> Adagrad, RMSprop
# 모멘텀 최적화와 RMSprop의 장점을 접목한 것 ->  Adam

In [14]:
import keras
# Adam클래스의 매개변수 기본값을 사용해 모델 훈련
model = keras.Sequential()
model.add(keras.layers.Input(shape=(28,28)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [15]:
# compile() 메서드의 optimizer를 'adam'으로 설정하고 5번의 에포크 동안 훈련
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 4s 2ms/step - loss: 0.5274 - accuracy: 0.8155
Epoch 2/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3950 - accuracy: 0.8575
Epoch 3/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3542 - accuracy: 0.8709
Epoch 4/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3259 - accuracy: 0.8804
Epoch 5/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3077 - accuracy: 0.8871


In [16]:
# 검증세트에서 성능 확인
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 1s 1ms/step - loss: 0.3620 - accuracy: 0.8664


[0.36196908354759216, 0.8664166927337646]